In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

from ai.FeatureCNN import FeatureCNN
from ai.DataLoader2 import DataLoader
from env import *

ETA = 1e-2
EPOCHS = 1000

print(torch.cuda.is_available())

True


In [2]:
train_dloader = DataLoader(TRAIN_DATA_PATH, TRASH_CAT, noise=True)
valid_dloader = DataLoader(VALID_DATA_PATH, TRASH_CAT, noise=True)

Number of data: 1241
Number of batch: 22
Number of data: 112
Number of batch: 2


In [3]:
model = FeatureCNN()
model = model.cuda()

In [4]:
optimizer = optim.Adam(model.parameters(), lr=ETA)
criterion = nn.NLLLoss()

In [6]:
model.load(CNN_CKPT_PATH)

top_valid_acc = 0.0

for e in range(EPOCHS):
    
    train_loss = 0.0
    train_clf_acc = 0.0
    
    for x_batch_, y_batch_ in train_dloader.next_batch():
        x_batch_ = x_batch_.reshape(-1, IN_CHANNEL, HEIGHT, WIDTH)
        y_batch = np.repeat(y_batch_, 8, axis=0)
        
        x_batch = torch.FloatTensor(x_batch_).cuda()
        y_batch = torch.LongTensor(y_batch).cuda()
        
        latent, logps = model(x_batch)
        loss = criterion(logps, y_batch)
        train_loss += loss.item()
        
        with torch.no_grad():
            ps = torch.exp(logps)
            val_k, cls_k = ps.topk(1, dim=1)
            equal = cls_k == y_batch.view(*cls_k.size())
            train_clf_acc += torch.mean(equal.type(torch.FloatTensor))
            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (e+1) % 10 == 0:
        
        valid_loss = 0.0
        valid_clf_acc = 0.0
        
        with torch.no_grad():
            model.eval()
            
            for x_batch_, y_batch_ in valid_dloader.next_batch():
                x_batch_ = x_batch_.reshape(-1, IN_CHANNEL, HEIGHT, WIDTH)
                y_batch = np.repeat(y_batch_, 8, axis=0)
                
                x_batch = torch.FloatTensor(x_batch_).cuda()
                y_batch = torch.LongTensor(y_batch).cuda()

                latent, logps = model(x_batch)
                ps = torch.exp(logps)
                _, topk = ps.topk(1, dim=1)
                equal = topk == y_batch.view(*topk.size())
                valid_clf_acc += torch.mean(equal.type(torch.FloatTensor))
        
                loss = criterion(logps, y_batch)
                valid_loss += loss.item()

            train_loss /= len(train_dloader)
            train_clf_acc /= len(train_dloader)
            valid_loss /= len(valid_dloader)
            valid_clf_acc /= len(valid_dloader)

            print(f"Epochs: {e+1}/{EPOCHS}")
            print(f"Train loss: {train_loss:.8f}")
            print(f"Train acc: {train_clf_acc:.8f}")
            print(f"Valid loss: {valid_loss:.8f}")
            print(f"Valid acc: {valid_clf_acc:.8f}")
            
            if top_valid_acc < valid_clf_acc:
                top_valid_acc = valid_clf_acc
                model.save(CNN_CKPT_PATH)

            model.train()
        
# model.save(CNN_CKPT_PATH)

FeatureCNN was loaded.
Epochs: 10/1000
Train loss: 0.35078301
Train acc: 0.87804377
Valid loss: 0.78606856
Valid acc: 0.72991073
FeatureCNN was saved.
Epochs: 20/1000
Train loss: 0.29991663
Train acc: 0.89752448
Valid loss: 0.75320953
Valid acc: 0.77455354
FeatureCNN was saved.
Epochs: 30/1000
Train loss: 0.27735517
Train acc: 0.90198857
Valid loss: 1.09572306
Valid acc: 0.72991073
Epochs: 40/1000
Train loss: 0.25947390
Train acc: 0.91071433
Valid loss: 0.86553314
Valid acc: 0.73437500
Epochs: 50/1000
Train loss: 0.24848677
Train acc: 0.91375810
Valid loss: 0.95542255
Valid acc: 0.75892854
Epochs: 60/1000
Train loss: 0.25629111
Train acc: 0.91112018
Valid loss: 1.05608356
Valid acc: 0.73214287
Epochs: 70/1000
Train loss: 0.21370514
Train acc: 0.92370129
Valid loss: 1.19883418
Valid acc: 0.75892860


KeyboardInterrupt: 